# Feature Engineering and train, dev, val split

In [8]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split 
from scipy.stats import mode
from sklearn.metrics import mean_squared_error 

%matplotlib inline
import gc #garbage collector interface

import warnings 
warnings.filterwarnings('ignore')

## Feature Engineering on the Historical and New Merchants Datasets

read Historical Transactions dataset

In [9]:
hist_df = pd.read_csv("data/historical_transactions.csv")
hist_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [10]:
hist_df.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id'],
      dtype='object')

read new merchants dataset

In [11]:
new_merchant_df = pd.read_csv("data/new_merchant_transactions.csv")
new_merchant_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [12]:
new_merchant_df.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id'],
      dtype='object')

define simple function to compute the [mode](https://en.wikipedia.org/wiki/Mode_(statistics)) of an array-like structure. The default scipy function returns both the value and the occurance absolute frequence.

In [13]:
def mod(arr):
    return mode(arr)[0][0]

We want to aggregate different function to different column of the dataframe. We construct three dictionaries with the following structure:
```python
{col_to_be_applied: {result_col_name: function}}
```


In [14]:
aggregationDictHist = {'card_id':{'hist_transactions_count':'count'},
        'merchant_category_id':{'hist_most_frequent_merchant_cat':mod},
                       'subsector_id':{'hist_most_frequent_subsector':mod},
        'city_id':{'hist_most_frequent_city':mod}, 'state_id':{'hist_most_frequent_state':mod},
        'month_lag':{'hist_min_month_lag':'min'},'purchase_amount':{'hist_max_purchase_amount':'max'}}

In [15]:
aggregationDictNew = {'card_id':{'new_transactions_count':'count'},
        'merchant_category_id':{'new_most_frequent_merchant_cat':mod},
                       'subsector_id':{'new_most_frequent_subsector':mod},
        'city_id':{'new_most_frequent_city':mod}, 'state_id':{'new_most_frequent_state':mod},
        'month_lag':{'new_min_month_lag':'min'},'purchase_amount':{'new_max_purchase_amount':'max'}}

apply aggregated functions to historical df

In [16]:
hist_grouped = hist_df[['card_id','merchant_category_id','subsector_id','city_id',
                'state_id','month_lag','purchase_amount']].groupby('card_id').agg(aggregationDictHist)
hist_grouped.columns = hist_grouped.columns.droplevel(0)

In [17]:
hist_grouped.head()

,hist_transactions_count,hist_most_frequent_merchant_cat,hist_most_frequent_subsector,hist_most_frequent_city,hist_most_frequent_state,hist_min_month_lag,hist_max_purchase_amount
card_id,,,,,,,
C_ID_00007093c1,149,307,19,244,2,-12,1.507069
C_ID_0001238066,123,307,19,314,9,-5,0.768095
C_ID_0001506ef0,66,705,33,137,19,-13,1.493545
C_ID_0001793786,216,278,37,179,-1,-9,4.554145
C_ID_000183fdda,144,367,16,161,3,-6,2.764788


In [18]:
#in hist dataset
totalH = hist_grouped.isnull().sum().sort_values(ascending = False)
percentH = (hist_grouped.isnull().sum()/hist_grouped.isnull().count()*100).sort_values(ascending = False)
missing_data_h  = pd.concat([totalH, percentH], axis=1, keys=['Total', 'Percent']) # axis = 1 because it concatenates 2 columns, not rows
missing_data_h.head(2)

,Total,Percent
hist_max_purchase_amount,0,0.0
hist_min_month_lag,0,0.0


Apply aggregated functions in the new merchants dataset:

In [19]:
new_merchant_grouped = new_merchant_df[['card_id','merchant_category_id','subsector_id','city_id',
                'state_id','month_lag','purchase_amount']].groupby('card_id').agg(aggregationDictNew)
new_merchant_grouped.columns = new_merchant_grouped.columns.droplevel(0)

In [20]:
new_merchant_grouped.head()

,new_transactions_count,new_most_frequent_merchant_cat,new_most_frequent_subsector,new_most_frequent_city,new_most_frequent_state,new_min_month_lag,new_max_purchase_amount
card_id,,,,,,,
C_ID_00007093c1,2,222,21,69,2,2,-0.656749
C_ID_0001238066,26,278,37,314,9,1,-0.078318
C_ID_0001506ef0,2,360,33,137,19,1,-0.715352
C_ID_0001793786,31,278,37,69,9,1,3.129932
C_ID_000183fdda,11,367,16,161,3,1,-0.107680


In [21]:
totalN = new_merchant_grouped.isnull().sum().sort_values(ascending = False)
percentN = (new_merchant_grouped.isnull().sum()/new_merchant_grouped.isnull().count()*100).sort_values(ascending = False)
missing_data_n  = pd.concat([totalN, percentN], axis=1, keys=['Total', 'Percent']) # axis = 1 because it concatenates 2 columns, not rows
missing_data_n.head(2)

,Total,Percent
new_max_purchase_amount,0,0.0
new_min_month_lag,0,0.0


## Train-val-dev split

We want to split the training set into train, dev, val, following a 60-20-20 splitting

In [22]:
train_df = pd.read_csv(os.path.join("data","train.csv"), 
                       parse_dates = ["first_active_month"]) # parse_dates param in order to have a correct date format

In [23]:
train_df.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056


In [24]:
test_df = pd.read_csv("data/test.csv", parse_dates = ["first_active_month"])

In [25]:
len(train_df)

201917

merge the hist and new merchants features to the train_df

In [26]:
train_df_feats = hist_grouped.reset_index().merge(new_merchant_grouped.reset_index(),
                                                 on='card_id').merge(train_df,on='card_id',how='right')

In [27]:
train_df_feats.head()

,card_id,hist_transactions_count,hist_most_frequent_merchant_cat,hist_most_frequent_subsector,hist_most_frequent_city,hist_most_frequent_state,hist_min_month_lag,hist_max_purchase_amount,new_transactions_count,new_most_frequent_merchant_cat,new_most_frequent_subsector,new_most_frequent_city,new_most_frequent_state,new_min_month_lag,new_max_purchase_amount,first_active_month,feature_1,feature_2,feature_3,target
0,C_ID_00007093c1,149.0,307.0,19.0,244.0,2.0,-12.0,1.507069,2.0,222.0,21.0,69.0,2.0,2.0,-0.656749,2017-02-01,5,1,1,0.134077
1,C_ID_0001506ef0,66.0,705.0,33.0,137.0,19.0,-13.0,1.493545,2.0,360.0,33.0,137.0,19.0,1.0,-0.715352,2016-07-01,2,2,0,0.884295
2,C_ID_000183fdda,144.0,367.0,16.0,161.0,3.0,-6.0,2.764788,11.0,367.0,16.0,161.0,3.0,1.0,-0.107680,2017-09-01,4,1,0,-0.264433
3,C_ID_0002c7c2c1,67.0,178.0,27.0,69.0,9.0,-9.0,-0.296112,6.0,108.0,27.0,188.0,9.0,1.0,-0.497768,2017-04-01,3,2,1,1.288514
4,C_ID_0003be3c83,171.0,705.0,37.0,279.0,9.0,-6.0,1.134802,6.0,78.0,29.0,279.0,9.0,1.0,-0.167936,2017-08-01,2,3,0,-0.083268


In [28]:
len(train_df_feats)

201917

In [29]:
train_df_feats.columns

Index(['card_id', 'hist_transactions_count', 'hist_most_frequent_merchant_cat',
       'hist_most_frequent_subsector', 'hist_most_frequent_city',
       'hist_most_frequent_state', 'hist_min_month_lag',
       'hist_max_purchase_amount', 'new_transactions_count',
       'new_most_frequent_merchant_cat', 'new_most_frequent_subsector',
       'new_most_frequent_city', 'new_most_frequent_state',
       'new_min_month_lag', 'new_max_purchase_amount', 'first_active_month',
       'feature_1', 'feature_2', 'feature_3', 'target'],
      dtype='object')

In [30]:
transform = [col for col in train_df_feats.columns if 'most' in col]

In [31]:
len(train_df_feats) - train_df_feats.count()

card_id                                0
hist_transactions_count            21931
hist_most_frequent_merchant_cat    21931
hist_most_frequent_subsector       21931
hist_most_frequent_city            21931
hist_most_frequent_state           21931
hist_min_month_lag                 21931
hist_max_purchase_amount           21931
new_transactions_count             21931
new_most_frequent_merchant_cat     21931
new_most_frequent_subsector        21931
new_most_frequent_city             21931
new_most_frequent_state            21931
new_min_month_lag                  21931
new_max_purchase_amount            21931
first_active_month                     0
feature_1                              0
feature_2                              0
feature_3                              0
target                                 0
dtype: int64

In [32]:
len(train_df_feats)

201917

In [33]:
train_df_feats.dropna(inplace=True)

In [34]:
len(train_df_feats)

179986

In [35]:
for col in transform:
    train_df_feats[col] = train_df_feats[col].map(lambda x: np.int(x))

In [36]:
# transform the first_activ_month feature in : number of months until 01.01.2019
import datetime as dt

reference_date = dt.datetime.strptime('Jan 1 2019', '%b %d %Y').date()
def months_between_dates(date2):
    return reference_date.month - date2.month + 12*(reference_date.year - date2.year)

In [37]:
train_df_feats['first_active_month'] = train_df_feats['first_active_month'].map(months_between_dates)

In [38]:
train_df_feats.set_index('card_id',inplace=True)

In [39]:
categorical = transform + ['feature_1', 'feature_2', 'feature_3']

In [40]:
train_df_feats.head(5)

,hist_transactions_count,hist_most_frequent_merchant_cat,hist_most_frequent_subsector,hist_most_frequent_city,hist_most_frequent_state,hist_min_month_lag,hist_max_purchase_amount,new_transactions_count,new_most_frequent_merchant_cat,new_most_frequent_subsector,new_most_frequent_city,new_most_frequent_state,new_min_month_lag,new_max_purchase_amount,first_active_month,feature_1,feature_2,feature_3,target
card_id,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,149.0,307,19,244,2,-12.0,1.507069,2.0,222,21,69,2,2.0,-0.656749,23,5,1,1,0.134077
C_ID_0001506ef0,66.0,705,33,137,19,-13.0,1.493545,2.0,360,33,137,19,1.0,-0.715352,30,2,2,0,0.884295
C_ID_000183fdda,144.0,367,16,161,3,-6.0,2.764788,11.0,367,16,161,3,1.0,-0.107680,16,4,1,0,-0.264433
C_ID_0002c7c2c1,67.0,178,27,69,9,-9.0,-0.296112,6.0,108,27,188,9,1.0,-0.497768,21,3,2,1,1.288514
C_ID_0003be3c83,171.0,705,37,279,9,-6.0,1.134802,6.0,78,29,279,9,1.0,-0.167936,17,2,3,0,-0.083268


In [41]:
import copy
# used : https://stackoverflow.com/questions/18016495/get-subset-of-most-frequent-dummy-variables-in-pandas
# func that returns a dummified DataFrame of significant dummies in a given column
def dum_sign(dummy_col, threshold=0.1):

    # removes the bind
    dummy_col = copy.copy(dummy_col)

    # what is the ratio of a dummy in whole column
    count = pd.value_counts(dummy_col) / len(dummy_col)

    # cond whether the ratios is higher than the threshold
    mask = dummy_col.isin(count[count > threshold].index)

    # replace the ones which ratio is lower than the threshold by a special name
    dummy_col[~mask] = "others"

    return pd.get_dummies(dummy_col, prefix=dummy_col.name)

In [42]:
print(train_df_feats.shape)
aux_df = train_df_feats.copy() # makes a deep copy of the df
aux_df.head()

for i in range(0, (train_df_feats.shape[1] - 1)):
    aux_df = aux_df.join(dum_sign(train_df_feats.iloc[:,i]))
    
print(aux_df.shape)
aux_df.head()

train_df_feats = aux_df

(179986, 19)
(179986, 63)


In [43]:
rest_df, val_df_feats = train_test_split(train_df_feats, test_size=0.2, random_state=23)

train_df_feats_final, dev_df_feats = train_test_split(rest_df, test_size=0.2, random_state=23)

In [44]:
print('length of training set: ', len(train_df_feats_final))
print('length of validation set: ', len(val_df_feats))
print('length of development set: ', len(dev_df_feats))
print('sum of the above: ',len(train_df_feats_final) + len(val_df_feats) + len(dev_df_feats))
print('length of previous trainingset: ',len(train_df))

length of training set:  115190
length of validation set:  35998
length of development set:  28798
sum of the above:  179986
length of previous trainingset:  201917


In [45]:
train_df_feats_final.to_csv(os.path.join('data','train_feats.csv'),index=False)
val_df_feats.to_csv(os.path.join('data','val_feats.csv'),index=False)
dev_df_feats.to_csv(os.path.join('data','dev_feats.csv'),index=False)

In [46]:
test_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04-01,C_ID_0ab67a22ab,3,3,1
1,2017-01-01,C_ID_130fd0cbdd,2,3,0
2,2017-08-01,C_ID_b709037bc5,5,1,1
3,2017-12-01,C_ID_d27d835a9f,2,1,0
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1


In [47]:
len(test_df)

123623

merge the hist and new merchants features to the train_df

In [48]:
test_df_feats = hist_grouped.reset_index().merge(new_merchant_grouped.reset_index(),
                                                 on='card_id').merge(test_df,on='card_id',how='right')

In [49]:
test_df_feats.head()

,card_id,hist_transactions_count,hist_most_frequent_merchant_cat,hist_most_frequent_subsector,hist_most_frequent_city,hist_most_frequent_state,hist_min_month_lag,hist_max_purchase_amount,new_transactions_count,new_most_frequent_merchant_cat,new_most_frequent_subsector,new_most_frequent_city,new_most_frequent_state,new_min_month_lag,new_max_purchase_amount,first_active_month,feature_1,feature_2,feature_3
0,C_ID_0001238066,123.0,307.0,19.0,314.0,9.0,-5.0,0.768095,26.0,278.0,37.0,314.0,9.0,1.0,-0.078318,2017-09-01,4,1,0
1,C_ID_0001793786,216.0,278.0,37.0,179.0,-1.0,-9.0,4.554145,31.0,278.0,37.0,69.0,9.0,1.0,3.129932,2017-01-01,5,2,1
2,C_ID_0002709b5a,73.0,705.0,33.0,103.0,18.0,-13.0,-0.336684,3.0,605.0,2.0,69.0,9.0,1.0,-0.611669,2015-08-01,5,1,1
3,C_ID_000298032a,30.0,560.0,34.0,233.0,9.0,-11.0,2.577843,1.0,432.0,27.0,23.0,9.0,2.0,-0.641872,2016-12-01,2,2,0
4,C_ID_0002ba3c2e,70.0,705.0,33.0,344.0,18.0,-9.0,-0.097011,3.0,705.0,33.0,344.0,18.0,1.0,-0.667883,2016-06-01,5,2,1


In [50]:
len(test_df_feats)

123623

In [ ]:
test_df_feats["first_active_month"] = test_df_feats["first_active_month"].map(months_between_dates) 
test_df_feats.set_index('card_id',inplace=True)


In [60]:
test_df_feats.dropna(inplace = True)
test_df_feats.shape

(110015, 18)

In [59]:
test_df_feats.to_csv(os.path.join('data','test_feats.csv'),index=False)